In [ ]:
from langchain_ollama import ChatOllama
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatOllama(
    model="llama3.2:1b",
    temperature=0.9, 
    top_p=0.95, 
    num_ctx=2048, 
    repeat_penalty=1.2
)
#Define a new graph
workflow = StateGraph(state_schema=MessagesState)
#define a function that calls model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages" : response}

#define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable" : {"thread_id" : "thread_1"}}

query = "Who was the first president of the United States?"
input_messages = [HumanMessage(content = query)]

for chunk, metadata in app.stream({"messages": input_messages}, config=config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
      print(chunk.content, end = "", flush=True)


George Washington was the first President of the United States. He served from April 30, 1789, to March 4, 1797.